# CS440/ECE448 Spring 2023
# MP11: Reinforcement Learning

The first thing you need to do is to download this file: <a href="mp11.zip">mp11.zip</a>.  If you want, you can also download <a href="mp11_extra.zip">mp11_extra.zip</a>, the extra credit assignment.  `mp11.zip` has the following content:

* `submitted.py`: Your homework. Edit, and then submit to <a href="https://www.gradescope.com/courses/486387">Gradescope</a>.
* `mp11_notebook.ipynb`: This is a <a href="https://anaconda.org/anaconda/jupyter">Jupyter</a> notebook to help you debug.  You can completely ignore it if you want, although you might find that it gives you useful instructions.
* `pong.py`: This is a program that plays Pong.  If called interactively, it will call the module `pong_display.py` to create a display, so that you can play.  If told to use a Q-learner, it will call your `submitted.py` to do Q-learning.
* `grade.py`: Once your homework seems to be working, you can test it by typing `python grade.py`, which will run the tests in `tests/tests_visible.py`.
* `tests/test_visible.py`: This file contains about half of the <a href="https://docs.python.org/3/library/unittest.html">unit tests</a> that Gradescope will run in order to grade your homework.  If you can get a perfect score on these tests, then you should also get a perfect score on the additional hidden tests that Gradescope uses.
* `requirements.txt`: This tells you which python packages you need to have installed, in order to run `grade.py`.  You can install all of those packages by typing `pip install -r requirements.txt` or `pip3 install -r requirements.txt`.

This file (`mp11_notebook.ipynb`) will walk you through the whole MP, giving you instructions and debugging tips as you go.

### Table of Contents

1. <a href="#section1">Playing Pong</a>
1. <a href="#section2">Creating a Q-Learner Object</a>
1. <a href="#section3">Epsilon-First Exploration</a>
1. <a href="#section4">Q-Learning</a>
1. <a href="#section5">Saving and Loading Your Q and N Tables</a>
1. <a href="#section6">Exploitation</a>
1. <a href="#section7">Acting</a>
1. <a href="#section8">Training</a>
1. <a href="#section9">Extra Credit</a>
1. <a href="#grade">Grade Your Homework</a>

<a id='section1'></a>

## Playing Pong

Pong was the <a href="https://en.wikipedia.org/wiki/Pong">first video game produced by Atari.</a>  It is a simple game, based on table tennis.  Here is a two-person version of the game: https://commons.wikimedia.org/wiki/File:Pong_Game_Test2.gif

We will be playing a one-person version of the game:

* When the ball hits the top, bottom, or left wall of the playing field, it bounces.
* The right end of the playing field is open, except for the paddle.  If the ball hits the paddle, it bounces, and the player's score increments by one.  If the ball hits the open space, the game is over; the score resets to zero, and a new game begins.

The game is pretty simple, but in order to get a better feeling for it, you may want to try playing it yourself.  Use the up arrow to move the paddle upward, and the down arrow to move the paddle downward.  See how high you can make your score:


In [1]:
# !python pong.py

Once you figure out how to use the arrow keys to control your paddle, we hope you will find that the game is not too hard for a human to play.  However, for a computer, it's difficult to know: where should the paddle be moved at each time step?  In order to see how difficult it is for a computer to play, let's ask the "random" player to play the game.

**WARNING:** The following line will open a pygame window.  The pygame window will be hidden by this window -- in order to see it, you will need to minimize this window.  The pygame window will consume a lot of CPU time just waiting for the processor, so in order to kill it, you will need to come back to this window, click on the block below, then click the Jupyter "stop" button (the square button at the top of this window) in order to stop processing.

In [2]:
# !python pong.py --player random

<a id='section2'></a>

## Creating a Q-Learner Object

The first thing you will do is to create a `q_learner` object that can store your learned Q table and your N table (table of exploration counts).  

Like any other object-oriented language, python permits you to create new object classes in order to store data that will be needed from time to time.  If you are not already very, very familiar with python classes, you might want to study the python class tutorial: https://docs.python.org/3/tutorial/classes.html

Like any other object in python, a `q_learner` object is created by calling its name as a function, e.g., `my_q_learner=submitted.q_learner()`.  Doing so calls the function `submitted.q_learner.__init__()`.  Let's look at the docstring to see what it should do.

In [3]:
import submitted, importlib
importlib.reload(submitted)
help(submitted.q_learner.__init__)

Help on function __init__ in module submitted:

__init__(self, alpha, epsilon, gamma, nfirst, state_cardinality)
    Create a new q_learner object.
    
    @params:
    alpha (scalar) - learning rate of the Q-learner
    epsilon (scalar) - probability of taking a random action
    gamma (scalar) - discount factor        
    nfirst (scalar) - exploring each state/action pair nfirst times before exploiting
    state_cardinality (list) - cardinality of each of the quantized state variables
    
    @return:
    None



Write your `__init__` function to meet the requirements specified in the docstring.  Once you have completed it, the following code should run without errors:

In [4]:
importlib.reload(submitted)

q_learner = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])

print(q_learner)


In [5]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_local()
    print("After local assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After nonlocal assignment: nonlocal spam
After local assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


<a id='section3'></a>

## Epsilon-First Exploration

In order to manage the exploration/exploitation tradeoff, we will be using both "epsilon-first" and "epsilon-greedy" (https://en.wikipedia.org/wiki/Multi-armed_bandit#Semi-uniform_strategies).  

The epsilon-first strategy explores every state/action pair at least `nfirst` times before it ever starts to exploit any strategy.  Your `q_learner` should have a table to keep track of how many times it has explored a state/action pair prior to the start of any exploitation.  The method for storing that table is up to you; in order to have some standardized API, therefore, you need to write a method called `report_exploration_counts` that returns a list of the three exploration counts for a given state. 

In [6]:
importlib.reload(submitted)
help(submitted.q_learner.report_exploration_counts)

Help on function report_exploration_counts in module submitted:

report_exploration_counts(self, state)
    Check to see how many times each action has been explored in this state.
    @params:
    state (list of 5 ints): ball_x, ball_y, ball_vx, ball_vy, paddle_y.
      These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
      and the y-position of the paddle, all quantized.
      0 <= state[i] < state_cardinality[i], for all i in [0,4].
    
    @return:
    explored_count (array of 3 ints): 
      number of times that each action has been explored from this state.
      The mapping from actions to integers is up to you, but there must be three of them.



Write `report_exploration_counts` so that it returns a list or array for any given state.  Test your code with the following:

In [7]:
importlib.reload(submitted)
q_learner = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])
print('This is how many times state [0,0,0,0,0] has been explored so far:')
print(q_learner.report_exploration_counts([0,0,0,0,0]))
print('This is how many times state [9,9,1,1,9] has been explored so far:')
print(q_learner.report_exploration_counts([9,9,1,1,9]))

This is how many times state [0,0,0,0,0] has been explored so far:
[0. 0. 0.]
This is how many times state [9,9,1,1,9] has been explored so far:
[0. 0. 0.]


When your learner first starts learning, it will call the function `choose_unexplored_action` to choose an unexplored action.  This function should choose a function uniformly at random from the set of unexplored actions in a given state, if there are any:

In [8]:
importlib.reload(submitted)
help(submitted.q_learner.choose_unexplored_action)

Help on function choose_unexplored_action in module submitted:

choose_unexplored_action(self, state)
    Choose an action that has been explored less than nfirst times.
    If many actions are underexplored, you should choose uniformly
    from among those actions; don't just choose the first one all
    the time.
    
    @params:
    state (list of 5 ints): ball_x, ball_y, ball_vx, ball_vy, paddle_y.
       These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
      and the y-position of the paddle, all quantized.
      0 <= state[i] < state_cardinality[i], for all i in [0,4].
    
    @return:
    action (scalar): either -1, or 0, or 1, or None
      If all actions have been explored at least n_explore times, return None.
      Otherwise, choose one uniformly at random from those w/count less than n_explore.
      When you choose an action, you should increment its count in your counter table.



If this has been written correctly, the following block should generate a random sequence of actions.  If the next block produces the same action 5 times in a row, that is the wrong result, and the result would be that your code does not pass the autograder.

In [9]:
importlib.reload(submitted)
q_learner = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))


Next action: -1
Next action: 0
Next action: 0
Next action: 1
Next action: 0


After all three actions have been explored `nfirst` times, the function `choose_unexplored_action` should return `None`, as shown here:

In [10]:
importlib.reload(submitted)
q_learner = submitted.q_learner(0.05,0.05,0.99,1,[10,10,2,2,10])
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))
print('Next action:',q_learner.choose_unexplored_action([9,9,1,1,9]))


Next action: -1
Next action: 1
Next action: 0
Next action: None


<a id='section4'></a>

## Q-Learning

The reinforcement learning we are implementing is called Q-learning (https://en.wikipedia.org/wiki/Q-learning).  

Q-learning keeps a table $Q[s,a]$ that specifies the expected utility of action $a$ in state $s$.  The organization of this table is up to you.  In order to have a standard API, the first thing you should implement is a function `report_q` with the following docstring:

In [11]:
importlib.reload(submitted)
help(submitted.q_learner.report_q)

Help on function report_q in module submitted:

report_q(self, state)
    Report the current Q values for the given state.
    @params:
    state (list of 5 ints): ball_x, ball_y, ball_vx, ball_vy, paddle_y.
      These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
      and the y-position of the paddle, all quantized.
      0 <= state[i] < state_cardinality[i], for all i in [0,4].
    
    @return:
    Q (array of 3 floats): 
      reward plus expected future utility of each of the three actions. 
      The mapping from actions to integers is up to you, but there must be three of them.



When your `q_learner` is first initialized, the value of $Q[state,action]$ should be zero for all state/action pairs, thus the `report_q` function should return lists of zeros:

In [12]:
importlib.reload(submitted)
q_learner=submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])
print('Q[0,0,0,0,0] is now:',q_learner.report_q([0,0,0,0,0]))
print('Q[9,9,1,1,9] is now:',q_learner.report_q([9,9,1,1,9]))

Q[0,0,0,0,0] is now: [0. 0. 0.]
Q[9,9,1,1,9] is now: [0. 0. 0.]


There are actually many different Q-learning algorithms available, but when people refer to **Q-learning with no modifier, they usually mean the time-difference (TD) algorithm**.  

- For example, this is the algorithm that's described on the wikipedia page (https://en.wikipedia.org/wiki/Q-learning).  

- This is the algorithm you will implement for this MP.

- In supervised machine learning, the learner tries to imitate a reference label.  
- In reinforcement learning, there is no reference label.  
- Q-learning replaces the reference label with a "local Q" value, which is the utility that was obtained by performing action $a$ in state $s$ one time.  

- It is usually calculated like this:

$$Q_{local}(s_t,a_t) = r_t + \gamma\max_{a_{t+1}}Q(s_{t+1},a_{t+1})$$

where $r_t$ is the reward that was achieved by performing action $a_t$ in state $s_t$, $s_{t+1}$ is the state into which the game transitioned, and $a_{t+1}$ is one of the actions that could be performed in that state.  $Q_{local}$ is computed by your `q_local` function, which has this docstring:  

In [13]:
importlib.reload(submitted)
help(submitted.q_learner.q_local)

Help on function q_local in module submitted:

q_local(self, reward, newstate)
    The update to Q estimated from a single step of game play:
    reward plus gamma times the max of Q[newstate, ...].
    
    @param:
    reward (scalar float): the reward achieved from the current step of game play.
    newstate (list of 5 ints): ball_x, ball_y, ball_vx, ball_vy, paddle_y.
      These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
      and the y-position of the paddle, all quantized.
      0 <= state[i] < state_cardinality[i], for all i in [0,4].
    
    @return:
    Q_local (scalar float): the local value of Q



Initially, `q_local` should just return the given reward, because initially, all Q values are 0:

In [14]:
importlib.reload(submitted)
q_learner = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,10,10])
print('Q_local(6.25,[9,9,1,1,9]) is currently:',q_learner.q_local(6.25,[9,9,1,1,9]))

Q_local(6.25,[9,9,1,1,9]) is currently: 6.25


Now you can use `q_learner.q_local` as the target for `q_learner.learn`.  The basic algorithm is

$$Q(s,a) = Q(s,a) + \alpha (Q_{local}(s,a)-Q(s,a))$$

Here is the docstring:

In [15]:
importlib.reload(submitted)
help(submitted.q_learner.learn)

Help on function learn in module submitted:

learn(self, state, action, reward, newstate)
    Update the internal Q-table on the basis of an observed
    state, action, reward, newstate sequence.
    
    @params:
    state: a list of 5 numbers: ball_x, ball_y, ball_vx, ball_vy, paddle_y.
      These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
      and the y-position of the paddle.
    action: an integer, one of -1, 0, or +1
    reward: a reward; positive for hitting the ball, negative for losing a game
    newstate: a list of 5 numbers, in the same format as state
    
    @return:
    None



The following block checks a sequence of Q updates:

1. First, $Q([9,9,1,1,9],-1)$ is updated.  Since all Q values start at zero, it will be updated to just have a value equal to $\alpha$ (0.05) times the given reward (6.25) for a total value of 0.3125.
1. When we print out $Q([9,9,1,1,9],:)$, we see that one of the elements has been updated.
1. Next, update $Q([9,9,1,1,8],1)$ with a given reward, and with $[9,9,1,1,9]$ as the given next state.  Since $Q([9,9,1,1,9],-1)$ is larger than zero, the next-state Q-value should be multiplied by $\gamma$ (0.99) and added to the reward (3.1), then multiplied by $\alpha$, giving a total value of 0.17046875.
1. The resulting Q-value is reported.

In [16]:
importlib.reload(submitted)
q_learner = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])
q_learner.learn([9,9,1,1,9],-1,6.25,[0,0,0,0,0])
print('Q[9,9,1,1,9] is now',q_learner.report_q([9,9,1,1,9]))
q_learner.learn([9,9,1,1,8],1,3.1,[9,9,1,1,9])
print('Q[9,9,1,1,8] is now',q_learner.report_q([9,9,1,1,8]))

Q[9,9,1,1,9] is now [0.3125 0.     0.    ]
Q[9,9,1,1,8] is now [0.         0.         0.17046875]


<a id='section5'></a>

## Saving and Loading your Q and N Tables

After you've spent a long time training your `q_learner`, you will want to save your Q and N tables so that you can reload them later.  The format of Q and N is up to you, therefore it's also up to you to write the `save` and `load` functions.  Here are the docstrings:

In [17]:
importlib.reload(submitted)
help(submitted.q_learner.save)

Help on function save in module submitted:

save(self, filename)
    Save your Q and N tables to a file.
    This can save in any format you like, as long as your "load" 
    function uses the same file format.  We recommend numpy.savez,
    but you can use something else if you prefer.
    
    @params:
    filename (str) - filename to which it should be saved
    @return:
    None



In [18]:
importlib.reload(submitted)
help(submitted.q_learner.load)

Help on function load in module submitted:

load(self, filename)
    Load the Q and N tables from a file.
    This should load from whatever file format your save function
    used.  We recommend numpy.load, but you can use something
    else if you prefer.
    
    @params:
    filename (str) - filename from which it should be loaded
    @return:
    None



These functions can be tested by doing one step of training one `q_learner`, then saving its results, then loading them into another `q_learner`:

In [19]:
importlib.reload(submitted)
q_learner1 = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])
print('Next action:',q_learner1.choose_unexplored_action([9,9,1,1,9]))
q_learner1.learn([9,9,1,1,9],-1,6.25,[0,0,0,0,0])
print('N1[9,9,1,1,8] is now',q_learner1.report_exploration_counts([9,9,1,1,9]))
print('Q1[9,9,1,1,8] is now',q_learner1.report_q([9,9,1,1,9]))
q_learner1.save('test.npz')

q_learner2 = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])
print('N2[9,9,1,1,8] starts out as',q_learner2.report_exploration_counts([9,9,1,1,9]))
print('Q2[9,9,1,1,8] starts out as',q_learner2.report_q([9,9,1,1,9]))
q_learner2.load('test.npz')
print('N2[9,9,1,1,8] is now',q_learner2.report_exploration_counts([9,9,1,1,9]))
print('Q2[9,9,1,1,8] is now',q_learner2.report_q([9,9,1,1,9]))


Next action: 1
N1[9,9,1,1,8] is now [0. 0. 1.]
Q1[9,9,1,1,8] is now [0.3125 0.     0.    ]
N2[9,9,1,1,8] starts out as [0. 0. 0.]
Q2[9,9,1,1,8] starts out as [0. 0. 0.]
N2[9,9,1,1,8] is now [0. 0. 1.]
Q2[9,9,1,1,8] is now [0.3125 0.     0.    ]


<a id='section6'></a>

## Exploitation

A reinforcement learner always has to trade off between exploration (choosing an action at random) versus exploitation (choosing the action with the maximum expected utility).  Before we worry about that tradeoff, though, let's first make sure that exploitation works.

In [20]:
importlib.reload(submitted)
help(submitted.q_learner.exploit)

Help on function exploit in module submitted:

exploit(self, state)
    Return the action that has the highest Q-value for the current state, and its Q-value.
    @params:
    state (list of 5 ints): ball_x, ball_y, ball_vx, ball_vy, paddle_y.
      These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
      and the y-position of the paddle, all quantized.
      0 <= state[i] < state_cardinality[i], for all i in [0,4].
    
    @return:
    action (scalar int): either -1, or 0, or 1.
      The action that has the highest Q-value.  Ties can be broken any way you want.
    Q (scalar float): 
      The Q-value of the selected action



In [21]:
importlib.reload(submitted)
q_learner1 = submitted.q_learner(0.05,0.05,0.99,5,[10,10,2,2,10])
q_learner1.learn([9,9,1,1,9],-1,6.25,[0,0,0,0,0])
print('Q1[9,9,1,1,9] is now',q_learner1.report_q([9,9,1,1,9]))
print('The best action and Q from state [9,9,1,1,9] are',q_learner1.exploit([9,9,1,1,9]))

Q1[9,9,1,1,9] is now [0.3125 0.     0.    ]
The best action and Q from state [9,9,1,1,9] are (-1, 0.3125)


<a id='section7'></a>

## Acting

When your learner decides which action to perform, it should trade off exploration vs. exploitation using both the epsilon-first and the epsilon-greedy strategies:
1. If there is any action that has been explored fewer than `nfirst` times, then choose one of those actions at random.  Otherwise...
1. With probability `epsilon`, choose an action at random.  Otherwise...
1. Exploit.


In [22]:
importlib.reload(submitted)
help(submitted.q_learner.act)

Help on function act in module submitted:

act(self, state)
    Decide what action to take in the current state.
    
    - If any action has been taken less than nfirst times, 
      - then choose one of those actions, uniformly at random.
    
    - Otherwise, with probability epsilon, choose an action uniformly at random.
    
    - Otherwise, choose the action with the best Q(state,action).
    
    @params: 
    state: a list of 5 integers: ball_x, ball_y, ball_vx, ball_vy, paddle_y.
      These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
      and the y-position of the paddle, all quantized.
      0 <= state[i] < state_cardinality[i], for all i in [0,4].
    
    @return:
    -1 if the paddle should move upward
    0 if the paddle should be stationary
    1 if the paddle should move downward



In [23]:
 
import random
 
# 生成第一个随机数
print( "random() : ", random.random())
 
# 生成第二个随机数
print( "random() : ", random.random())

random() :  0.25861000858143524
random() :  0.7864804938233956


In order to test all three types of action (epsilon-first exploration, epsilon-greedy exploration, and exploitation), let's create a learner with `nfirst=1` and `epsilon=0.25`, and set it so that the best action from state `[9,9,1,1,9]` is `-1`.  With these settings, a sequence of calls to `q_learner.act` should produce the following sequence of actions:

1. The first three actions should include each possible action once.
1. After the first three actions, 3/4 of the remaining actions should be `-1`.  The remaining 1/4 should be randomly chosen.

In [24]:
importlib.reload(submitted)
q_learner=submitted.q_learner(0.05,0.25,0.99,1,[10,10,2,2,10])
q_learner.learn([9,9,1,1,9],-1,6.25,[0,0,0,0,0])
print('An epsilon-first action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-first action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-first action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))
print('An epsilon-greedy explore/exploit action:',q_learner.act([9,9,1,1,9]))

An epsilon-first action: 0
An epsilon-first action: -1
An epsilon-first action: 1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: 0
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1
An epsilon-greedy explore/exploit action: -1


<a id='section8'></a>

## Training

Now that all of your components work, you can try training your algorithm.  Do this by giving your `q_learner` as a player to a new `pong.PongGame` object.  Set `visibility=False` so that the `PongGame` doesn't create a new window.

In [25]:
import pong, importlib, submitted
importlib.reload(pong)
help(pong.PongGame.__init__)

Help on function __init__ in module pong:

__init__(self, ball_speed=4, paddle_speed=8, learner=None, visible=True, state_quantization=[10, 10, 2, 2, 10])
    Create a new pong game, with a specified player.
    
    @params:
    ball_speed (scalar int) - average ball speed in pixels/frame
    paddle_speed (scalar int) - paddle moves 0, +paddle_speed, or -paddle_speed
    learner - can be None if the player is human.  If not None, should be an 
      object of type random_learner, submitted.q_learner, or submitted.deep_q.
    visible (bool) - should this game have an attached pygame window?
    state_quantization (list) - if not None, state variables are quantized
      into integers of these cardinalities before being passed to the learner.



As you can see, we should set `visibility=False` so that the `PongGame` doesn't create a new window.  We should also make sure that the PongGame uses the same state quantization as the learner.

In [26]:
importlib.reload(pong)
importlib.reload(submitted)
state_quantization = [10,10,2,2,10]
q_learner=submitted.q_learner(0.05,0.05,0.99,5,state_quantization)

pong_game = pong.PongGame(learner=q_learner, visible=False, state_quantization=state_quantization)
print(pong_game)

In order to train our learner, we want it to play the game many times.  To do that we use the PongGame.run function:

In [27]:
help(pong_game.run)


Help on method run in module pong:

run(m_rewards=inf, m_games=inf, m_frames=inf, states=[]) method of pong.PongGame instance
    Run the game.
    @param
    m_frames (scalar int): maximum number of frames to be played
    m_rewards (scalar int): maximum number of rewards earned (+ or -)
    m_games (scalar int): maximum number of games
    states (list): list of states whose Q-values should be returned
       each state is a list of 5 ints: ball_x, ball_y, ball_vx, ball_vy, paddle_y.
       These are the (x,y) position of the ball, the (vx,vy) velocity of the ball,
       and the y-position of the paddle, all quantized.
       0 <= state[i] < state_cardinality[i], for all i in [0,4].
    
      
    @return
    scores (list): list of scores of all completed games
    
    The following will be returned only if the player is q_learning or deep_q.
    New elements will be added to these lists once/frame if m_frames is specified,
    else once/reward if m_rewards is specified, else once

In order to make sure our learner is learning, let's tell `pong_game.run` to output all 3 Q-values of all of the 4000 states in every time step.

To make sure that's not an outrageous amount of data, let's tell it to only output the Q values once/reward, and ask it to only collect 5000 rewards:


In [28]:
states = [[x,y,vx,vy,py] for x in range(10) for y in range(10) for vx in range(2) for vy in range(2) for py in range(10) ]

scores, q_achieved, q_states = pong_game.run(m_rewards=500, states=states)
q_learner.save('trained_model.npz')

print('The number of games played was',len(scores))
print('The number of rewards was',len(q_states))
print('The size of each returned Q-matrix was',q_states[0].shape)


Completed 0 games, 2 rewards, 735 frames, score 1, max score 1
Completed 1 games, 3 rewards, 943 frames, score 0, max score 1
Completed 2 games, 4 rewards, 1151 frames, score 0, max score 1
Completed 3 games, 5 rewards, 1405 frames, score 0, max score 1
Completed 4 games, 7 rewards, 1956 frames, score 1, max score 1
Completed 5 games, 8 rewards, 2210 frames, score 0, max score 1
Completed 6 games, 9 rewards, 2391 frames, score 0, max score 1
Completed 7 games, 10 rewards, 2645 frames, score 0, max score 1
Completed 8 games, 11 rewards, 2853 frames, score 0, max score 1
Completed 9 games, 12 rewards, 3061 frames, score 0, max score 1
Completed 10 games, 13 rewards, 3269 frames, score 0, max score 1
Completed 11 games, 16 rewards, 4916 frames, score 2, max score 2
Completed 12 games, 17 rewards, 5170 frames, score 0, max score 2
Completed 13 games, 18 rewards, 5424 frames, score 0, max score 2
Completed 14 games, 19 rewards, 5678 frames, score 0, max score 2
Completed 15 games, 20 reward

Completed 126 games, 173 rewards, 41374 frames, score 0, max score 3
Completed 127 games, 174 rewards, 41582 frames, score 0, max score 3
Completed 128 games, 176 rewards, 42712 frames, score 1, max score 3
Completed 129 games, 178 rewards, 43083 frames, score 1, max score 3
Completed 130 games, 179 rewards, 43291 frames, score 0, max score 3
Completed 131 games, 181 rewards, 43803 frames, score 1, max score 3
Completed 132 games, 182 rewards, 44057 frames, score 0, max score 3
Completed 133 games, 183 rewards, 44265 frames, score 0, max score 3
Completed 134 games, 184 rewards, 44446 frames, score 0, max score 3
Completed 135 games, 185 rewards, 44627 frames, score 0, max score 3
Completed 136 games, 186 rewards, 44835 frames, score 0, max score 3
Completed 137 games, 187 rewards, 45043 frames, score 0, max score 3
Completed 138 games, 189 rewards, 45395 frames, score 1, max score 3
Completed 139 games, 190 rewards, 45603 frames, score 0, max score 3
Completed 140 games, 198 rewards, 

Completed 245 games, 417 rewards, 107246 frames, score 3, max score 7
Completed 246 games, 420 rewards, 107953 frames, score 2, max score 7
Completed 247 games, 422 rewards, 108367 frames, score 1, max score 7
Completed 248 games, 424 rewards, 108734 frames, score 1, max score 7
Completed 249 games, 425 rewards, 108988 frames, score 0, max score 7
Completed 250 games, 430 rewards, 109904 frames, score 4, max score 7
Completed 251 games, 431 rewards, 110085 frames, score 0, max score 7
Completed 252 games, 433 rewards, 110452 frames, score 1, max score 7
Completed 253 games, 434 rewards, 110706 frames, score 0, max score 7
Completed 254 games, 436 rewards, 111257 frames, score 1, max score 7
Completed 255 games, 438 rewards, 111801 frames, score 1, max score 7
Completed 256 games, 439 rewards, 111982 frames, score 0, max score 7
Completed 257 games, 440 rewards, 112236 frames, score 0, max score 7
Completed 258 games, 442 rewards, 112733 frames, score 1, max score 7
Completed 259 games,

The returned value of `q_states` is a list of 4000x3 numpy arrays (20 states, 3 actions).  The list contains `m_rewards` of these. We want to convert it into something that matplotlib can plot.  

In [29]:
import numpy as np

Q = np.array([np.reshape(q,-1) for q in q_states])
print('Q is now of shape',Q.shape)
print('the max absolute value of Q is ',np.amax(abs(Q)))

Q is now of shape (500, 12000)
the max absolute value of Q is  1.8012977192492898


In [30]:
# %matplotlib inline

# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(14,6),layout='tight')
# ax = [ fig.add_subplot(2,1,x) for x in range(1,3) ]
# ax[0].plot(np.arange(0,len(q_states)),Q)
# ax[0].set_title('Q values of all states')
# ax[1].plot(np.arange(0,len(q_states)),q_achieved)
# ax[1].set_title('Q values of state achieved at each time')
# ax[1].set_ylabel('Reward number')

OK, now let's try running it for a much longer period -- say, 5000 complete games.  We won't ask it to print out any states this time.

In [31]:
scores, q_achieved, q_states = pong_game.run(m_games=5000, states=[])

print('The number of games played was',len(scores))
print('The number of video frames was',len(q_states))
print('The size of each returned Q-matrix was',q_states[0].shape)


Completed 0 games, 1 rewards, 182 frames, score 0, max score 7
Completed 1 games, 2 rewards, 390 frames, score 0, max score 7
Completed 2 games, 3 rewards, 644 frames, score 0, max score 7
Completed 3 games, 4 rewards, 825 frames, score 0, max score 7
Completed 4 games, 6 rewards, 1285 frames, score 1, max score 7
Completed 5 games, 8 rewards, 1653 frames, score 1, max score 7
Completed 6 games, 10 rewards, 2131 frames, score 1, max score 7
Completed 7 games, 11 rewards, 2312 frames, score 0, max score 7
Completed 8 games, 13 rewards, 2809 frames, score 1, max score 7
Completed 9 games, 16 rewards, 3774 frames, score 2, max score 7
Completed 10 games, 17 rewards, 3982 frames, score 0, max score 7
Completed 11 games, 21 rewards, 4758 frames, score 3, max score 7
Completed 12 games, 23 rewards, 5145 frames, score 1, max score 7
Completed 13 games, 24 rewards, 5353 frames, score 0, max score 7
Completed 14 games, 26 rewards, 5790 frames, score 1, max score 7
Completed 15 games, 27 rewards

Completed 140 games, 294 rewards, 70779 frames, score 1, max score 7
Completed 141 games, 296 rewards, 71441 frames, score 1, max score 7
Completed 142 games, 298 rewards, 72175 frames, score 1, max score 7
Completed 143 games, 299 rewards, 72429 frames, score 0, max score 7
Completed 144 games, 302 rewards, 73447 frames, score 2, max score 7
Completed 145 games, 304 rewards, 73788 frames, score 1, max score 7
Completed 146 games, 306 rewards, 74194 frames, score 1, max score 7
Completed 147 games, 309 rewards, 75058 frames, score 2, max score 7
Completed 148 games, 310 rewards, 75266 frames, score 0, max score 7
Completed 149 games, 312 rewards, 75634 frames, score 1, max score 7
Completed 150 games, 315 rewards, 76399 frames, score 2, max score 7
Completed 151 games, 316 rewards, 76653 frames, score 0, max score 7
Completed 152 games, 317 rewards, 76834 frames, score 0, max score 7
Completed 153 games, 318 rewards, 77042 frames, score 0, max score 7
Completed 154 games, 319 rewards, 

Completed 282 games, 630 rewards, 154289 frames, score 3, max score 12
Completed 283 games, 634 rewards, 155123 frames, score 3, max score 12
Completed 284 games, 635 rewards, 155331 frames, score 0, max score 12
Completed 285 games, 639 rewards, 156104 frames, score 3, max score 12
Completed 286 games, 640 rewards, 156285 frames, score 0, max score 12
Completed 287 games, 641 rewards, 156466 frames, score 0, max score 12
Completed 288 games, 642 rewards, 156647 frames, score 0, max score 12
Completed 289 games, 646 rewards, 157491 frames, score 3, max score 12
Completed 290 games, 651 rewards, 159728 frames, score 4, max score 12
Completed 291 games, 654 rewards, 160606 frames, score 2, max score 12
Completed 292 games, 657 rewards, 161138 frames, score 2, max score 12
Completed 293 games, 658 rewards, 161319 frames, score 0, max score 12
Completed 294 games, 664 rewards, 162150 frames, score 5, max score 12
Completed 295 games, 665 rewards, 162358 frames, score 0, max score 12
Comple

Completed 414 games, 955 rewards, 236384 frames, score 1, max score 12
Completed 415 games, 959 rewards, 237314 frames, score 3, max score 12
Completed 416 games, 970 rewards, 239288 frames, score 10, max score 12
Completed 417 games, 974 rewards, 240001 frames, score 3, max score 12
Completed 418 games, 980 rewards, 241004 frames, score 5, max score 12
Completed 419 games, 992 rewards, 243617 frames, score 11, max score 12
Completed 420 games, 994 rewards, 244219 frames, score 1, max score 12
Completed 421 games, 998 rewards, 244925 frames, score 3, max score 12
Completed 422 games, 999 rewards, 245179 frames, score 0, max score 12
Completed 423 games, 1003 rewards, 246175 frames, score 3, max score 12
Completed 424 games, 1006 rewards, 246655 frames, score 2, max score 12
Completed 425 games, 1009 rewards, 247191 frames, score 2, max score 12
Completed 426 games, 1011 rewards, 247925 frames, score 1, max score 12
Completed 427 games, 1013 rewards, 248437 frames, score 1, max score 12

Completed 556 games, 1449 rewards, 356128 frames, score 16, max score 16
Completed 557 games, 1453 rewards, 357109 frames, score 3, max score 16
Completed 558 games, 1458 rewards, 358041 frames, score 4, max score 16
Completed 559 games, 1463 rewards, 358906 frames, score 4, max score 16
Completed 560 games, 1464 rewards, 359087 frames, score 0, max score 16
Completed 561 games, 1467 rewards, 359577 frames, score 2, max score 16
Completed 562 games, 1472 rewards, 360605 frames, score 4, max score 16
Completed 563 games, 1476 rewards, 361506 frames, score 3, max score 16
Completed 564 games, 1479 rewards, 362047 frames, score 2, max score 16
Completed 565 games, 1482 rewards, 362754 frames, score 2, max score 16
Completed 566 games, 1487 rewards, 363758 frames, score 4, max score 16
Completed 567 games, 1491 rewards, 364721 frames, score 3, max score 16
Completed 568 games, 1496 rewards, 366862 frames, score 4, max score 16
Completed 569 games, 1499 rewards, 367815 frames, score 2, max 

...

Completed 4552 games, 40106 rewards, 9932470 frames, score 4, max score 144
Completed 4553 games, 40135 rewards, 9937171 frames, score 28, max score 144
Completed 4554 games, 40155 rewards, 9941916 frames, score 19, max score 144
Completed 4555 games, 40157 rewards, 9942413 frames, score 1, max score 144
Completed 4556 games, 40162 rewards, 9943572 frames, score 4, max score 144
Completed 4557 games, 40192 rewards, 9948061 frames, score 29, max score 144
Completed 4558 games, 40201 rewards, 9949759 frames, score 8, max score 144
Completed 4559 games, 40217 rewards, 9952970 frames, score 15, max score 144
Completed 4560 games, 40233 rewards, 9955503 frames, score 15, max score 144
Completed 4561 games, 40242 rewards, 9957015 frames, score 8, max score 144
Completed 4562 games, 40244 rewards, 9957452 frames, score 1, max score 144
Completed 4563 games, 40253 rewards, 9958980 frames, score 8, max score 144
Completed 4564 games, 40259 rewards, 9960173 frames, score 5, max score 144
Complet

Completed 4659 games, 41606 rewards, 10290070 frames, score 24, max score 144
Completed 4660 games, 41616 rewards, 10292397 frames, score 9, max score 144
Completed 4661 games, 41622 rewards, 10293348 frames, score 5, max score 144
Completed 4662 games, 41626 rewards, 10294172 frames, score 3, max score 144
Completed 4663 games, 41631 rewards, 10295074 frames, score 4, max score 144
Completed 4664 games, 41645 rewards, 10299030 frames, score 13, max score 144
Completed 4665 games, 41663 rewards, 10304078 frames, score 17, max score 144
Completed 4666 games, 41668 rewards, 10305726 frames, score 4, max score 144
Completed 4667 games, 41670 rewards, 10306109 frames, score 1, max score 144
Completed 4668 games, 41676 rewards, 10308759 frames, score 5, max score 144
Completed 4669 games, 41686 rewards, 10310761 frames, score 9, max score 144
Completed 4670 games, 41692 rewards, 10311679 frames, score 5, max score 144
Completed 4671 games, 41703 rewards, 10313515 frames, score 10, max score

Completed 4770 games, 42710 rewards, 10576640 frames, score 13, max score 144
Completed 4771 games, 42721 rewards, 10578616 frames, score 10, max score 144
Completed 4772 games, 42724 rewards, 10579308 frames, score 2, max score 144
Completed 4773 games, 42726 rewards, 10579706 frames, score 1, max score 144
Completed 4774 games, 42734 rewards, 10581331 frames, score 7, max score 144
Completed 4775 games, 42764 rewards, 10587685 frames, score 29, max score 144
Completed 4776 games, 42776 rewards, 10589578 frames, score 11, max score 144
Completed 4777 games, 42780 rewards, 10590252 frames, score 3, max score 144
Completed 4778 games, 42785 rewards, 10591300 frames, score 4, max score 144
Completed 4779 games, 42791 rewards, 10592616 frames, score 5, max score 144
Completed 4780 games, 42796 rewards, 10593569 frames, score 4, max score 144
Completed 4781 games, 42807 rewards, 10595902 frames, score 10, max score 144
Completed 4782 games, 42809 rewards, 10596339 frames, score 1, max scor

Completed 4877 games, 43563 rewards, 10826913 frames, score 19, max score 144
Completed 4878 games, 43564 rewards, 10827094 frames, score 0, max score 144
Completed 4879 games, 43573 rewards, 10830025 frames, score 8, max score 144
Completed 4880 games, 43578 rewards, 10831482 frames, score 4, max score 144
Completed 4881 games, 43584 rewards, 10834296 frames, score 5, max score 144
Completed 4882 games, 43598 rewards, 10840784 frames, score 13, max score 144
Completed 4883 games, 43606 rewards, 10842095 frames, score 7, max score 144
Completed 4884 games, 43611 rewards, 10843097 frames, score 4, max score 144
Completed 4885 games, 43613 rewards, 10843759 frames, score 1, max score 144
Completed 4886 games, 43616 rewards, 10844296 frames, score 2, max score 144
Completed 4887 games, 43619 rewards, 10844843 frames, score 2, max score 144
Completed 4888 games, 43631 rewards, 10848448 frames, score 11, max score 144
Completed 4889 games, 43634 rewards, 10849025 frames, score 2, max score 

Completed 4987 games, 44277 rewards, 11011530 frames, score 17, max score 144
Completed 4988 games, 44293 rewards, 11013897 frames, score 15, max score 144
Completed 4989 games, 44298 rewards, 11014671 frames, score 4, max score 144
Completed 4990 games, 44302 rewards, 11015485 frames, score 3, max score 144
Completed 4991 games, 44309 rewards, 11016939 frames, score 6, max score 144
Completed 4992 games, 44317 rewards, 11019249 frames, score 7, max score 144
Completed 4993 games, 44326 rewards, 11021126 frames, score 8, max score 144
Completed 4994 games, 44336 rewards, 11023373 frames, score 9, max score 144
Completed 4995 games, 44338 rewards, 11023810 frames, score 1, max score 144
Completed 4996 games, 44344 rewards, 11025506 frames, score 5, max score 144
Completed 4997 games, 44359 rewards, 11028657 frames, score 14, max score 144
Completed 4998 games, 44364 rewards, 11029637 frames, score 4, max score 144
Completed 4999 games, 44379 rewards, 11033561 frames, score 14, max score

Now let's plot the score, to see if it improved over time.  We will also plot the local average, averaged over 10 consecutive games, to see if that has improved.  Notice that we can use `np.convolve` to compute the local average.

These numbers are really noisy, with a really large maximum.  We will plot `np.log10(1+x)`, rather than x, so that we can better see the average numbers, and ignore the very large noisy spikes.

In [ ]:
# %matplotlib inline

# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(14,9),layout='tight')
# ax = [ fig.add_subplot(3,1,x) for x in range(1,4) ]
# ax[0].plot(np.arange(0,len(scores)),np.log10(1+np.array(scores)))
# ax[0].plot([0,5000],np.log10([7,7]),'k--')
# ax[0].set_title('Game scores')
# ax[1].plot(np.arange(4991),np.log10(1+np.convolve(np.ones(10)/10,scores,mode='valid')))
# ax[1].plot([0,4991],np.log10([7,7]),'k--')
# ax[1].set_title('Game scores, average 10 consecutive games')
# ax[2].plot(np.arange(0,len(q_achieved)),q_achieved)
# ax[2].set_title('Q values of state achieved at each time')
# ax[2].set_ylabel('Game number')

NameError: name 'ax' is not defined

In [ ]:
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import numpy as np
# fig, ax = plt.subplots()  # Create a figure containing a single axes.
# ax.plot([1, 2, 3, 4], [1, 4, 2, 3])  # Plot some data on the axes.

Hooray, it has learned!  If you are getting a ten-game average score of better than 6, then you are ready to submit your model for grading.   In order to do that, you need to save the model:

In [ ]:
# q_learner.save('trained_model.npz')

In [ ]:
# !python pong.py --player q_trained

<a id='section9'></a>

## Extra Credit

For extra credit, download the file <a href="mp11_extra.zip">mp11_extra.zip</a>.  The only important file in this package is:
* `tests/test_extra.py`: this contains extra tests that will evaluate your pre-trained `deep_q` learner, which should be in a file called `trained_model.pkl`.  For full credit, your model should achieve an average score of greater than 20, averaged over 10 consecutive games. 

With a quantized lookup table, it's probably not possible to achieve an average score of 20.  With a deep-Q learner, however, it is eminently possible.  In order to do the extra credit, therefore, you should just fill in the part of `submitted.py` that implements the `deep_q` learner, using pytorch to define a model structure, train it, save it, load it, and act on it.  This learner only needs to have five methods: `__init__`, `act`, `learn`, `save`, and `load`:

<a id='grade'></a>

In [19]:
import submitted, importlib,pong
importlib.reload(pong)
importlib.reload(submitted)
state_quantization = [10,10,2,2,10]
deep_q=submitted.deep_q(0.05,0.05,0.99,5)

pong_game = pong.PongGame(learner=deep_q, visible=False, state_quantization=None)
print(pong_game)

In [21]:
importlib.reload(submitted)
states = [[x,y,vx,vy,py] for x in range(10) for y in range(10) for vx in range(2) for vy in range(2) for py in range(10) ]

scores, q_achieved, q_states = pong_game.run(m_rewards=500, states=states)
deep_q.save("trained_model.pkl")

print('The number of games played was',len(scores))
print('The number of rewards was',len(q_states))
print('The size of each returned Q-matrix was',q_states[0].shape)


Completed 0 games, 1 rewards, 209 frames, score 0, max score 3
Completed 1 games, 2 rewards, 417 frames, score 0, max score 3
Completed 2 games, 3 rewards, 671 frames, score 0, max score 3
Completed 3 games, 5 rewards, 1142 frames, score 1, max score 3
Completed 4 games, 6 rewards, 1350 frames, score 0, max score 3
Completed 5 games, 7 rewards, 1604 frames, score 0, max score 3
Completed 6 games, 8 rewards, 1785 frames, score 0, max score 3
Completed 7 games, 9 rewards, 2039 frames, score 0, max score 3
Completed 8 games, 10 rewards, 2293 frames, score 0, max score 3
Completed 9 games, 11 rewards, 2501 frames, score 0, max score 3
Completed 10 games, 12 rewards, 2755 frames, score 0, max score 3
Completed 11 games, 13 rewards, 2936 frames, score 0, max score 3
Completed 12 games, 15 rewards, 3448 frames, score 1, max score 3
Completed 13 games, 19 rewards, 4318 frames, score 3, max score 3
Completed 14 games, 21 rewards, 4755 frames, score 1, max score 3
Completed 15 games, 22 rewards,

ValueError: too many values to unpack (expected 3)

## Grade your homework

If you've reached this point, and all of the above sections work, then you're ready to try grading your homework!  Before you submit it to Gradescope, try grading it on your own machine.  This will run some visible test cases (which you can read in `tests/test_visible.py`), and compare the results to the solutions (which you can read in `solution.json`).

The exclamation point (!) tells python to run the following as a shell command.  Obviously you don't need to run the code this way -- this usage is here just to remind you that you can also, if you wish, run this command in a terminal window.

In [11]:
!python grade.py

EEEEE
ERROR: test_deep_q_exceeds_12 (test_extra.TestStep)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "d:\# Courses\## 大三\SP23\CS440\mp11_extra\tests\test_extra.py", line 24, in test_deep_q_exceeds_12
    scores, q_achieved, q_states = pong_game.run(m_games=10, states=[])
  File "d:\# Courses\## 大三\SP23\CS440\mp11_extra\pong.py", line 202, in run
    self.learner.learn(state, action, reward, newstate)
  File "d:\# Courses\## 大三\SP23\CS440\mp11_extra\submitted.py", line 476, in learn
    label_pred = self.model(state)
  File "C:\Users\everloom\AppData\Roaming\Python\Python39\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "d:\# Courses\## 大三\SP23\CS440\mp11_extra\submitted.py", line 301, in forward
    y = self.pred(x)
  File "C:\Users\everloom\AppData\Roaming\Python\Python39\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    retur

Now you should try uploading your code to <a href="https://www.gradescope.com/courses/486387">Gradescope</a>.  

**Warning:** For this MP you need to update two files, not just one. 
* `trained_model.npz` should contain your trained model
* `submitted.py` should contain your code that loads and runs your trained model.

**Warning:** The autograder calculates the average score over ten random games.  If you are getting an average score above 10 almost all the time on your own computer, and if the autograder says you had a score below 10, try resubmitting to see if the next round of random games is better.

**Extra Credit:** Your extra credit should also be uploaded as two files,
* `trained_model.pkl` should contain your trained model
* `submitted.py` should contain your code that loads and runs your trained model.